# Support Vector Machine Regression 

## Mục tiêu
- Xây dựng mô hình SVM Regressor để dự đoán tuổi thọ trung bình
- Sử dụng dữ liệu đã được tiền xử lý từ `data/processed/`
- Tối ưu hóa siêu tham số bằng 5-Fold Cross-Validation
- Đánh giá mô hình trên tập train
- Lưu mô hình đã huấn luyện 

## Giới thiệu

SVR là phiên bản hồi quy của SVM, có khả năng mô hình hóa quan hệ phi tuyến bằng kernel.

### **Ưu điểm:**

- Mạnh mẽ trong dữ liệu phi tuyến.

- Không bị ảnh hưởng nhiều bởi outliers.

- Tổng quát hóa tốt.

### **Nhược điểm:**

- Chậm khi dữ liệu lớn.

- Cần chuẩn hóa dữ liệu.

- Khó chọn tham số (C, epsilon, gamma).

## Bước 1 - Import các thư viện cần thiết

### 1.1. Import thư viện

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.svm import SVR, LinearSVR
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import joblib

### 1.2. Cấu hình thư mục

In [2]:
RANDOM_STATE = 42
os.makedirs("../models/2_SVM_regression", exist_ok=True)

## Bước 2 - Đọc dữ liệu đã tiền xử lý 

In [3]:
# Đọc dữ liệu
train_df = pd.read_csv('../data/processed/train.csv')

print("THÔNG TIN DỮ LIỆU")
print("="*60)
print(f"Kích thước tập train: {train_df.shape}")

# Hiển thị 5 dòng đầu của tập train
print("\n5 dòng đầu tiên của tập train (đã được chuẩn hóa):")
train_df.head()

THÔNG TIN DỮ LIỆU
Kích thước tập train: (3255, 13)

5 dòng đầu tiên của tập train (đã được chuẩn hóa):


,country_name,country_code,year,population,pop_growth,life_expectancy,gdp_per_capita,gdp_growth,sanitation,electricity,water_access,co2_emissions,labor_force
0,Romania,ROU,2019,-0.100387,-1.101137,75.607317,-0.107887,0.079336,1.170060,0.621932,0.746494,-0.047183,-0.871164
1,Mauritius,MUS,2006,-0.235882,-0.500838,72.432195,-0.405891,0.241659,-0.060699,0.586683,0.718065,-0.221970,-0.189257
2,Angola,AGO,2008,-0.083899,1.500859,55.281000,-0.470953,1.327705,-0.060699,-1.545881,-2.268636,-0.432669,1.598947
3,Albania,ALB,2001,-0.222240,-1.349940,75.083000,-0.584364,0.930412,-1.071878,0.600783,-0.024271,-0.404910,-0.104497
4,Central African Republic,CAF,2021,-0.206911,0.236338,40.279000,-0.618434,-0.427187,-1.640089,-2.349558,-3.008624,-0.529029,1.172890


## Bước 3 - Chuẩn bị dữ liệu cho mô hình

Tách biến mục tiêu (`life_expectancy`) khỏi các đặc trưng. Loại bỏ các cột không cần thiết như `country_name`, `country_code`.

In [4]:
# Định nghĩa các cột dùng để dự đoán
feature_cols = [col for col in train_df.columns 
                if col not in ['life_expectancy', 'country_name', 'country_code']]

# Tách X và y cho từng tập
X_train = train_df[feature_cols]
y_train = train_df['life_expectancy']

print("THÔNG TIN CÁC TẬP DỮ LIỆU")
print("="*60)
print(f"Số lượng đặc trưng: {len(feature_cols)}")
print(f"\nCác đặc trưng được sử dụng:")
for i, col in enumerate(feature_cols, 1):
    print(f"  {i}. {col}")

print(f"\nKích thước X_train: {X_train.shape}")
print(f"Kích thước y_train: {y_train.shape}")

THÔNG TIN CÁC TẬP DỮ LIỆU
Số lượng đặc trưng: 10

Các đặc trưng được sử dụng:
  1. year
  2. population
  3. pop_growth
  4. gdp_per_capita
  5. gdp_growth
  6. sanitation
  7. electricity
  8. water_access
  9. co2_emissions
  10. labor_force

Kích thước X_train: (3255, 10)
Kích thước y_train: (3255,)


## Bước 4 - Xây dựng và huấn luyện mô hình LinearSVR bằng RandomizedSearchCV (k=5)


### 4.1. Tìm cấu hình tốt nhất cho LinearSVR bằng 5-fold CV trên tập train

In [5]:
from scipy.stats import uniform, randint

# Lưới tham số cho LinearSVR
param_distributions = {
    "C": uniform(0.01, 100),              # 0.01 → 100
    "epsilon": uniform(0.001, 1.0),       # 0.001 → 1.0
    "loss": ["epsilon_insensitive", "squared_epsilon_insensitive"],
    "max_iter": randint(1000, 10000),     # random số vòng lặp
}

# Tìm ra mô hình LinearSVR tốt nhất
linsvr = LinearSVR(random_state=RANDOM_STATE, dual='auto')
cv_linear_svr = RandomizedSearchCV(
    linsvr, 
    param_distributions=param_distributions, 
    scoring='neg_mean_squared_error', 
    cv=5, 
    n_jobs=-1, 
    verbose=2,
    n_iter=60
)

# Huấn luyện mô hình
cv_linear_svr.fit(X_train, y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


c:\Users\DELL\.conda\envs\course\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


,estimator,LinearSVR(random_state=42)
,param_distributions,"{'C': <scipy.stats....001E1D6D2DB90>, 'epsilon': <scipy.stats....001E1D6D2F750>, 'loss': ['epsilon_insensitive', 'squared_epsilon_insensitive'], 'max_iter': <scipy.stats....001E1D6D2FD50>}"
,n_iter,60
,scoring,'neg_mean_squared_error'
,n_jobs,-1
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,random_state,None
,error_score,nan


### 4.2. Hiển thị siêu tham số tối ưu

In [6]:
best_linsvr = cv_linear_svr.best_estimator_
best_params_linsvr = cv_linear_svr.best_params_

print("SIÊU THAM SỐ TỐI ƯU")
print("="*60)
for param, value in best_params_linsvr.items():
    print(f"  {param:20s}: {value}")

SIÊU THAM SỐ TỐI ƯU
  C                   : 16.658391969396472
  epsilon             : 0.7434417450862704
  loss                : epsilon_insensitive
  max_iter            : 9099


### 4.3. Đánh giá và lưu model

In [7]:
# Đánh giá trên tập train
y_pred_linsvr = best_linsvr.predict(X_train)
linsvr_rmse = np.sqrt(mean_squared_error(y_train, y_pred_linsvr))
linsvr_r2  = r2_score(y_train, y_pred_linsvr)
linsvr_mae = mean_absolute_error(y_train, y_pred_linsvr)

print("KẾT QUẢ MÔ HÌNH TỐI ƯU")
print("="*60)
print("Model: LinearSVR:")
print(f"RMSE loss: {linsvr_rmse:.3f}")
print(f"MAE loss: {linsvr_mae:.3f}")
print(f"R2 score: {linsvr_r2:.3f}")

# Lưu model
joblib.dump(best_linsvr, "../models/2_SVM_regression/linear_svr.pkl")

KẾT QUẢ MÔ HÌNH TỐI ƯU
Model: LinearSVR:
RMSE loss: 4.583
MAE loss: 3.353
R2 score: 0.721


['../models/2_SVM_regression/linear_svr.pkl']

## Bước 5 - Xây dựng và huấn luyện mô hình SVR (kernel=rbf) bằng RandomizedSearchCV (k=5)

### 5.1. Tìm cấu hình tốt nhất cho SVR bằng 5-fold CV 

In [8]:
from scipy.stats import loguniform, uniform

# Lưới tham số cho SVR (RBF)
param_distributions_svr_rbf = {
    "C": loguniform(1e-2, 1e3),           
    "gamma": loguniform(1e-4, 1e0),        
    "epsilon": uniform(0.001, 1.0),      
}

svr = SVR()
cv_svr = RandomizedSearchCV(
    svr, 
    param_distributions_svr_rbf, 
    scoring='neg_mean_squared_error', 
    cv=5, 
    n_jobs=-1, 
    verbose=2,
    n_iter=60
)

cv_svr.fit(X_train, y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


,estimator,SVR()
,param_distributions,"{'C': <scipy.stats....001E1D6D94A90>, 'epsilon': <scipy.stats....001E1D6D95D50>, 'gamma': <scipy.stats....001E1D0C04350>}"
,n_iter,60
,scoring,'neg_mean_squared_error'
,n_jobs,-1
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,random_state,None
,error_score,nan


### 5.2. Hiển thị siêu tham số tốt nhất của mô hình

In [9]:
best_svr = cv_svr.best_estimator_
best_params_svr = cv_svr.best_params_

print("SIÊU THAM SỐ TỐI ƯU")
print("="*60)
for param, value in best_params_svr.items():
    print(f"  {param:20s}: {value}")

SIÊU THAM SỐ TỐI ƯU
  C                   : 350.22762868788527
  epsilon             : 0.6066592489004907
  gamma               : 0.027546833440245288


### 5.2. Đánh giá và lưu model

In [10]:
y_pred_svr = best_svr.predict(X_train)
svr_rmse = np.sqrt(mean_squared_error(y_train, y_pred_svr))
svr_r2  = r2_score(y_train, y_pred_svr)
svr_mae = mean_absolute_error(y_train, y_pred_svr)

print("KẾT QUẢ MÔ HÌNH TỐI ƯU")
print("="*60)
print("Model: SVR:")
print(f"RMSE loss: {svr_rmse:.3f}")
print(f"MAE loss: {svr_mae:.3f}")
print(f"R2 score: {svr_r2:.3f}")

# Lưu model
joblib.dump(best_svr, "../models/2_SVM_regression/svr_rbf.pkl")

KẾT QUẢ MÔ HÌNH TỐI ƯU
Model: SVR:
RMSE loss: 2.695
MAE loss: 1.730
R2 score: 0.903


['../models/2_SVM_regression/svr_rbf.pkl']

## Kết luận
Thông qua notebook này, ta đã:
- Tìm được bộ tham số tốt nhất tìm được cho LinearSVR/SVR-RBF
- Train các mô hình này trên tập dữ liệu huấn luyện
- Đánh giá các mô hình thông qua các loại loss khác nhau: MAE, RMSE, R2